In [9]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import tensorflow as tf
from tensorflow.keras import regularizers
import numpy as np
import plotly.io

df = pd.read_excel('ANN_TRAIN_TEST_RAMON_T40_AIR_WATER.xlsx')
X = df.iloc[:,1:7].values
Y = df.iloc[:, [12,13]].values


#Parameter control
trainsize = 0.4
neurons = 6
activation_function = 'relu'
batchsize = 32
epochs = 250
hidden_layers_quantity = 3

#normalize data
from sklearn.preprocessing import StandardScaler
scalerx = StandardScaler()
scalery = StandardScaler()
X_scaled = scalerx.fit_transform(X)
Y_scaled = scalery.fit_transform(Y)

#split data  
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_scaled, Y_scaled, 
                                                train_size = trainsize, 
                                                random_state = 0)

In [10]:
#Building the ANN MODEL
from sklearn.metrics import r2_score, mean_squared_error

resultados = []

for neurons in range(1, neurons+1):

    for hidden_layers_quantity in range (1, hidden_layers_quantity+1):

        ann = tf.keras.models.Sequential() #Initializethe ann model

        #creatin a sequence of hidden layers (adding according to the range)
        for _ in range(hidden_layers_quantity):
            ann.add(tf.keras.layers.Dense(units=neurons, 
                                        activation=activation_function))                        
            ann.add(tf.keras.layers.Dropout(0.2)) #DROPUT LAYER

        ann.add(tf.keras.layers.Dense(units = 2)) # OUTPUT LAYER

        #Compiling the ann, it adds the stochastic gradient descent method to control losses
        ann.compile(optimizer = 'adam', 
                    loss = 'mean_squared_error') #COMPILER (TRY DIFFERENT PARAMETERS)

        #TRAIN THE ANN MODEL
        ann.fit(Xtrain, Ytrain, 
                batch_size = batchsize, 
                epochs = epochs)
        Qpred = ann.predict(Xtest)

        
        rmse = mean_squared_error(Ytest, Qpred, squared = False)
        r2 = r2_score(Ytest, Qpred)

        resultados.append((neurons, hidden_layers_quantity, rmse, r2))

        


Epoch 1/250
17/17 [==============================] - 0s 753us/step - loss: 1.3757
Epoch 2/250
17/17 [==============================] - 0s 751us/step - loss: 1.3514
Epoch 3/250
17/17 [==============================] - 0s 751us/step - loss: 1.2844
Epoch 4/250
17/17 [==============================] - 0s 751us/step - loss: 1.2218
Epoch 5/250
17/17 [==============================] - 0s 688us/step - loss: 1.2251
Epoch 6/250
17/17 [==============================] - 0s 688us/step - loss: 1.1664
Epoch 7/250
17/17 [==============================] - 0s 691us/step - loss: 1.1664
Epoch 8/250
17/17 [==============================] - 0s 691us/step - loss: 1.1222
Epoch 9/250
17/17 [==============================] - 0s 698us/step - loss: 1.1175
Epoch 10/250
17/17 [==============================] - 0s 688us/step - loss: 1.1019
Epoch 11/250
17/17 [==============================] - 0s 688us/step - loss: 1.0938
Epoch 12/250
17/17 [==============================] - 0s 688us/step - loss: 1.0811
Epoch 13/250


In [8]:
# O RMSE, QUANTO MENOR O VALOR, MELHOR.  PARA O R2, O MELHOR É 1.
for result in resultados:
    print(f"Para {result[0]} neurônios e {result[1]} camadas, RMSE = {result[2]}, R2 = {result[3]}")


Para 1 neurônios e 1 camadas, RMSE = 0.4351230753560468, R2 = 0.8107856134002072
Para 1 neurônios e 2 camadas, RMSE = 0.5742350938211295, R2 = 0.6708365245866241
Para 2 neurônios e 1 camadas, RMSE = 0.2115673134746433, R2 = 0.9553232815600929
Para 2 neurônios e 2 camadas, RMSE = 0.41132937305230644, R2 = 0.8310393787323453


In [ ]:
#apply inverse transform on the Xtest and Ytest
Xtest_original = scalerx.inverse_transform(Xtest)
Ytest_original = scalery.inverse_transform(Ytest)

#predict function
Qpred = ann.predict(Xtest)
Qpred_original = scalery.inverse_transform(Qpred)

In [ ]:
graf_exp_water = go.Scatter(x=X[:, 5], 
                            y=Y[:, 0], 
                            mode='markers', 
                            name='EXPERIMENTAL_WATER', 
                            marker=dict(size=10, 
                                        color='purple', 
                                        symbol='pentagon'))

Qtest_water = go.Scatter(x=Xtest_original[:, 5], 
                         y=Ytest_original[:, 0], 
                         mode='markers', 
                         name='Qtest_WATER',
                         marker=dict(size=11, 
                                     color='green', 
                                     symbol='x', 
                                     line={'width': 2}))

Qpred_water = go.Scatter(x=Xtest_original[:, 5], 
                         y=Qpred_original[:, 0], 
                         mode='markers', 
                         name='Qpred_WATER', 
                         marker=dict(size=7, 
                                     color='red', 
                                     symbol='circle'))

data_water = [graf_exp_water, Qtest_water, Qpred_water]

layout_water = go.Layout(title='ANN_TRAIN_TEST_RAMON_T40_AIR_WATER', 
                         xaxis=dict(title='Vazão massica do ar (m_ar)'), 
                         yaxis=dict(title='Taxa de troca térmica Q_water'))

fig_water = go.Figure(data_water, layout_water)

pyo.plot(fig_water)

